In [1]:
%%time
import tensorflow as tf
import tensorflow.contrib.slim as slim
import numpy as np
import pickle
import cv2
import os
import json
import sys
import lmdb
from collections import defaultdict
import random
from utils import *
from datetime import datetime

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,3" # GPU_ID
gpu_options = tf.GPUOptions(allow_growth=True)
config = tf.ConfigProto(gpu_options=gpu_options,log_device_placement=True,allow_soft_placement=True)

#############
#ToDo: add capabality of finding jpgs in parent even if ID.ltf.xml or ID.psm.xml is given
#############

working_path = '/home/bobby/aida_copy/AIDA/M18_copy/data/'
corpus_path = '/dvmm-filer2/projects/AIDA/data/ldc_eval_m18/LDC2019E42_AIDA_Phase_1_Evaluation_Source_Data_V1.0/'
# corpus_path = '/dvmm-filer2/projects/AIDA/data/ldc_isi_dryrun3/dryrun-updated_tmp/dryrun/'
#corpus_path = '/dvmm-filer2/projects/AIDA/data/ldc_isi_dryrun3/dryrun/' # D3
#corpus_path = '/dvmm-filer2/projects/AIDA/data/ldc_isi_dryrun/dryrun/' # D2

img_path = corpus_path
kfrm_path = corpus_path + 'data/video_shot_boundaries/representative_frames'
parent_child_tab = corpus_path + 'docs/parent_children.sorted.tab'
kfrm_msb = corpus_path + 'docs/masterShotBoundary.msb'
print('Check Point: Raw Data corpus_path change',corpus_path)


#rpi mention results in AIF
p_f = 'E'
video_asr_path = working_path + 'raw_files/RPI_TA1_'+p_f+'_asr/ltf_asr'
video_map_path = working_path + 'raw_files/RPI_TA1_'+p_f+'_asr/map_asr'
print('Check Point: RPI path change',video_asr_path)

det_results_path_img = working_path + 'objdet_results/'+p_f+'/det_results_merged_34a_jpg.pkl'
# det_results_path_img = working_path + 'objdet_results/det_results_merged_32_jpg.pkl'
det_results_path_kfrm = working_path + 'objdet_results/'+p_f+'/det_results_merged_34b_kf.pkl'
# det_results_path_kfrm = working_path + 'objdet_results/det_results_merged_33_kf.pkl'
print('Check Point: Alireza path change:','\n',det_results_path_img,'\n', det_results_path_kfrm,'\n')


Check Point: Raw Data corpus_path change /dvmm-filer2/projects/AIDA/data/ldc_eval_m18/LDC2019E42_AIDA_Phase_1_Evaluation_Source_Data_V1.0/
Check Point: RPI path change /home/bobby/aida_copy/AIDA/M18_copy/data/raw_files/RPI_TA1_E_asr/ltf_asr
Check Point: Alireza path change: 
 /home/bobby/aida_copy/AIDA/M18_copy/data/objdet_results/E/det_results_merged_34a_jpg.pkl 
 /home/bobby/aida_copy/AIDA/M18_copy/data/objdet_results/E/det_results_merged_34b_kf.pkl 

CPU times: user 2.38 s, sys: 672 ms, total: 3.06 s
Wall time: 27.6 s


In [2]:
%%time
#loading grounding pretrained model
print('Loading grounding pretrained model...')
model_path = working_path+ 'models/model_ELMo_PNASNET_VOA_norm'
sess, graph = load_model(model_path,config)
input_img = graph.get_tensor_by_name("input_img:0")
mode = graph.get_tensor_by_name("mode:0")
v = graph.get_tensor_by_name("image_local_features:0")
v_bar = graph.get_tensor_by_name("image_global_features:0")
print('Loading done.')

W0705 11:42:54.809000 140317059749632 deprecation_wrapper.py:119] From /home/bobby/aida_copy/AIDA/M18_copy/code/utils.py:874: The name tf.InteractiveSession is deprecated. Please use tf.compat.v1.InteractiveSession instead.



Loading grounding pretrained model...
Model Loading...


W0705 11:42:58.254809 140317059749632 deprecation_wrapper.py:119] From /home/bobby/aida_copy/AIDA/M18_copy/code/utils.py:875: The name tf.global_variables_initializer is deprecated. Please use tf.compat.v1.global_variables_initializer instead.

W0705 11:42:58.282377 140317059749632 deprecation_wrapper.py:119] From /home/bobby/aida_copy/AIDA/M18_copy/code/utils.py:875: The name tf.tables_initializer is deprecated. Please use tf.compat.v1.tables_initializer instead.

W0705 11:42:58.354227 140317059749632 deprecation_wrapper.py:119] From /home/bobby/aida_copy/AIDA/M18_copy/code/utils.py:877: The name tf.train.import_meta_graph is deprecated. Please use tf.compat.v1.train.import_meta_graph instead.



pass


W0705 11:43:04.331134 140317059749632 deprecation.py:323] From /dvmm-filer2/users/bobby/anaconda/envs/aida/lib/python3.6/site-packages/tensorflow/python/training/saver.py:1276: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to check for files with this prefix.


----Model Loaded-----
Loading done.
CPU times: user 7.79 s, sys: 4.06 s, total: 11.9 s
Wall time: 24.6 s


In [3]:
%%time
#preparing dicts
parent_dict, child_dict = create_dict(parent_child_tab)
id2dir_dict_kfrm = create_dict_kfrm(kfrm_path, kfrm_msb, video_asr_path, video_map_path)
#jpg
path_dict = create_path_dict(os.path.join(corpus_path,'data/jpg/jpg/'))
#mp4
path_dict.update(create_path_dict_kfrm(id2dir_dict_kfrm))
# print('HC000TJCP' in id2dir_dict_kfrm.keys())
# print(id2dir_dict_kfrm.keys())

#loading object detection results
with open(det_results_path_img, 'rb') as f:
    dict_obj_img = pickle.load(f)
    
with open(det_results_path_kfrm, 'rb') as f:
    dict_obj_kfrm = pickle.load(f)
print(datetime.now())
# print(child_dict)

2019-07-05 11:43:23.069239
CPU times: user 1.03 s, sys: 224 ms, total: 1.25 s
Wall time: 3.74 s


# Semantic Features

In [4]:
# about 8 hours in total for Instance Features
out_path_jpg = working_path + 'all_features/semantic_features_jpg.lmdb'
out_path_kfrm = working_path + 'all_features/semantic_features_keyframe.lmdb'

#opening lmdb environment
lmdb_env_jpg = lmdb.open(out_path_jpg, map_size=int(1e11), lock=False)
lmdb_env_kfrm = lmdb.open(out_path_kfrm, map_size=int(1e11), lock=False)

In [5]:
%%time 
#about 1.5 hour
print(datetime.now())
missed_children_jpg = []
for i, key in enumerate(dict_obj_img):
    imgs,_ = fetch_img(key+'.jpg.ldcc', parent_dict, child_dict, path_dict, level = 'Child')
    if len(imgs)==0:
        missed_children_jpg.append(key)
        continue
    img_batch, bb_ids, bboxes_norm = batch_of_bbox(imgs[0], dict_obj_img, key,\
                                        score_thr=0, filter_out=False)
    if len(bb_ids)>0:
        feed_dict = {input_img: img_batch, mode: 'test'}
        v_pred = sess.run([v], feed_dict)[0]
        for j,bb_id in enumerate(bb_ids):
            mask = mask_fm_bbox(feature_map_size=(19,19),bbox_norm=bboxes_norm[j,:],order='xyxy')
            if np.sum(mask)==0:
                continue
            img_vec = np.average(v_pred[j,:], weights = np.reshape(mask,[361]), axis=0)
            save_key = key+'/'+str(bb_id)
            with lmdb_env_jpg.begin(write=True) as lmdb_txn:
                lmdb_txn.put(save_key.encode(), img_vec)
    sys.stderr.write("Stored for image {} / {} \r".format(i, len(dict_obj_img)))
print(datetime.now())

2019-06-30 18:29:40.096989


2019-06-30 19:54:51.626702
CPU times: user 1d 9h 41s, sys: 2h 43min 43s, total: 1d 11h 44min 24s
Wall time: 1h 25min 11s


In [6]:
%%time 
#about 4-6 hours
print(datetime.now())
missed_children_kfrm = []
for i, key in enumerate(dict_obj_kfrm):
    # key+'.mp4.ldcc'
#     print('path from obj detecton for kfrm:',key+'.mp4.ldcc')
    imgs,_ = fetch_img(key+'.mp4.ldcc', parent_dict, child_dict, path_dict, level = 'Child')
    if len(imgs)==0:
        missed_children_kfrm.append(key)
        continue
    img_batch, bb_ids, bboxes_norm = batch_of_bbox(imgs[0], dict_obj_kfrm, key,\
                                      score_thr=0, filter_out=False)
    if len(bb_ids)>0:
        feed_dict = {input_img: img_batch, mode: 'test'}
        v_pred = sess.run([v], feed_dict)[0]
        for j,bb_id in enumerate(bb_ids):
            mask = mask_fm_bbox(feature_map_size=(19,19),bbox_norm=bboxes_norm[j,:],order='xyxy')
            if np.sum(mask)==0:
                continue
            img_vec = np.average(v_pred[j,:], weights = np.reshape(mask,[361]), axis=0)
            save_key = key+'/'+str(bb_id)
            with lmdb_env_kfrm.begin(write=True) as lmdb_txn:
                lmdb_txn.put(save_key.encode(), img_vec)
    sys.stderr.write("Stored for keyframe {} / {} \r".format(i, len(dict_obj_kfrm)))
print(datetime.now())

2019-06-30 19:54:51.636527


2019-07-01 02:08:46.869308
CPU times: user 5d 15h 9min 38s, sys: 11h 49s, total: 6d 2h 10min 27s
Wall time: 6h 13min 55s


In [7]:
len(missed_children_jpg)

0

In [8]:
len(missed_children_kfrm)

0

# Instance Features

In [4]:
# about 3 hours in total for Instance Features
out_path_jpg = working_path + 'all_features/instance_features_jpg.lmdb'
out_path_kfrm = working_path + 'all_features/instance_features_keyframe.lmdb'


#opening lmdb environment
lmdb_env_jpg = lmdb.open(out_path_jpg, map_size=int(1e11), lock=False)
lmdb_env_kfrm = lmdb.open(out_path_kfrm, map_size=int(1e11), lock=False)

In [5]:
%%time
#loading instance matching pretrained model
sess, graph = load_model(working_path+ 'models/model_universal_no_recons_ins_only',config)
input_img = graph.get_tensor_by_name("input_img:0")
mode = graph.get_tensor_by_name("mode:0")
img_vec = graph.get_tensor_by_name("img_vec:0")

Model Loading...
pass


/dvmm-filer2/users/bobby/anaconda/envs/aida/lib/python3.6/site-packages/tensorflow/python/client/session.py:1735: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


----Model Loaded-----
CPU times: user 5.16 s, sys: 8.03 s, total: 13.2 s
Wall time: 47.2 s


In [8]:
%%time 
#about 0.5 hour
print(datetime.now())
missed_children_jpg = []
for i, key in enumerate(dict_obj_img):
    # Todo test
    if 'HC0005KMS' not in key: #or 'HC0001H01' in key:
        continue
    print(i,key)

    imgs,_ = fetch_img(key+'.jpg.ldcc', parent_dict, child_dict, path_dict, level = 'Child')
    if len(imgs)==0:
        missed_children_jpg.append(key)
        continue
    img_batch, bb_ids, bboxes_norm = batch_of_bbox(imgs[0], dict_obj_img, key,\
                                        score_thr=0, filter_out=False,img_size=(224,224))
    
    if len(bb_ids)>0:
        # Test for Corpping bug
        feed_dict = {input_img: img_batch, mode: 'test'}
        img_vec_pred = sess.run([img_vec], feed_dict)[0]
#     print('img_batch',img_batch)
#         print('img_batch len:',len(img_batch),np.shape(img_batch))
#         print('img_batch vec:',img_batch)
#         print(np.shape(img_vec_pred))
#         print('img_vec_pred',type(img_vec_pred),img_vec_pred)
        for j,bb_id in enumerate(bb_ids):
            save_key = key+'/'+str(bb_id)
            with lmdb_env_jpg.begin(write=True) as lmdb_txn:
                lmdb_txn.put(save_key.encode(), img_vec_pred[j,:])
#                 print(sum(img_vec_pred[j,:]))
    sys.stderr.write("Stored for image {} / {} \r".format(i, len(dict_obj_img)))
print(datetime.now())

2019-07-05 12:04:15.054688
2019-07-05 12:04:15.056789
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 2.3 ms


In [7]:
%%time 
#about 3 hours
missed_children_kfrm = []
for i, key in enumerate(dict_obj_kfrm):
    imgs,_ = fetch_img(key+'.mp4.ldcc', parent_dict, child_dict, path_dict, level = 'Child')
    if len(imgs)==0:
        missed_children_kfrm.append(key)
        continue
    img_batch, bb_ids, bboxes_norm = batch_of_bbox(imgs[0], dict_obj_kfrm, key,\
                                      score_thr=0, filter_out=False,img_size=(224,224))
    if len(bb_ids)>0:
        feed_dict = {input_img: img_batch, mode: 'test'}
        img_vec_pred = sess.run([img_vec], feed_dict)[0]
        for j,bb_id in enumerate(bb_ids):
            save_key = key+'/'+str(bb_id)
            with lmdb_env_kfrm.begin(write=True) as lmdb_txn:
                lmdb_txn.put(save_key.encode(), img_vec_pred[j,:])
    sys.stderr.write("Stored for keyframe {} / {} \r".format(i, len(dict_obj_kfrm)))
print(datetime.now())

2019-06-30 17:58:58.874723
CPU times: user 1d 4h 47min 57s, sys: 3h 8min 51s, total: 1d 7h 56min 48s
Wall time: 1h 39min 49s


In [8]:
missed_children_jpg

[]

In [9]:
missed_children_kfrm

[]

In [10]:
print('finished')

finished
